# Задание

Требуется спроектировать и реализовать систему защиты паролей при их хранении в записных книжках или блокнотах.

In [4]:
import math
import random, sys

Вспомогательные функции

In [16]:
def egcd(a,b):
    '''
    Расширенный алгоритм Евклида
    Возвращает x, y, gcd(a,b) такие, что ax + by = gcd(a,b)
    '''
    u, u1 = 1, 0
    v, v1 = 0, 1
    while b:
        q = a // b
        u, u1 = u1, u - q * u1
        v, v1 = v1, v - q * v1
        a, b = b, a - q * b
    return u, v, a

def gcd(a,b):
    '''
    Работает быстрее, чем стандартный gcd
    '''
    a,b = (b,a) if a<b else (a,b)
    
    while b:
        a,b=b,a%b
    return a

def lcm(a, b):
    '''
    Подсчёт НОК
    '''
    return (a * b) // math.gcd(a, b)

def modInverse(e,n):
    '''
    Возвращает d: de = 1 (mod n)
    e должно быть взаимно просто с n
    '''
    return egcd(e,n)[0]%n

def simplenum(n):

    if n % 2 == 0:
        return n == 2
    d = 3
    while d * d <= n and n % d != 0:
        d += 2
    return d * d > n


def totient(p,q):
    
    return (p-1)*(q-1)

def bitlength(x):
    '''
    Вычисление битовой длины x
    '''
    assert x >= 0
    n = 0
    while x > 0:
        n = n+1
        x = x>>1
    return n


def sieve(n: int) -> list:
    """
    Sieve away and only primes are left.
    """
    primes = 2*[False] + (n-1)*[True]
    for i in range(2, int(n**0.5+1.5)):
        for j in range(i*i, n+1, i):
            primes[j] = False
    return [prime for prime, checked in enumerate(primes) if checked]


def isqrt(n):

    if n < 0:
        raise ValueError('Квадратный корень не вычисляется для отрицательных чисел')
    
    if n == 0:
        return 0
    a, b = divmod(bitlength(n), 2)
    x = 2**(a+b)
    while True:
        y = (x + n//x)//2
        if y >= x:
            return x
        x = y


def is_perfect_square(n):
    '''
    Возвращает квадратный корень, если число -- полный квадрат, 
    и -1 -- в противном случае
    '''
    h = n & 0xF; 
    
    if h > 9:
        return -1 

    if ( h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8 ):
        
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1


def rational_to_contfrac(x,y):
    '''
    Converts a rational x/y fraction into
    a list of partial quotients [a0, ..., an]
    '''
    a = x//y
    pquotients = [a]
    while a * y != x:
        x,y = y,x-a*y
        a = x//y
        pquotients.append(a)
    return pquotients


def convergents_from_contfrac(frac):
    '''
    computes the list of convergents
    using the list of partial quotients
    '''
    convs = [];
    for i in range(len(frac)):
        convs.append(contfrac_to_rational(frac[0:i]))
    return convs


def contfrac_to_rational (frac):
    '''Converts a finite continued fraction [a0, ..., an]
     to an x/y rational.
     '''
    if len(frac) == 0:
        return (0,1)
    num = frac[-1]
    denom = 1
    for _ in range(-2,-len(frac)-1,-1):
        num, denom = frac[_]*num+denom, num
    return (num,denom)

In [6]:
def test1():

    testnums = [(1, 1), (1, 2), (5, 15), (27, 73), (73, 27)]
    for r in testnums:
        (num, denom) = r
        print('rational number:')
        print(r)

        contfrac = rational_to_contfrac (num, denom)
        print('continued fraction:')
        print(contfrac)

        print('convergents:')
        print(convergents_from_contfrac(contfrac))
        print('***********************************')

In [7]:
test1()

rational number:
(1, 1)
continued fraction:
[1]
convergents:
[(0, 1)]
***********************************
rational number:
(1, 2)
continued fraction:
[0, 2]
convergents:
[(0, 1), (0, 1)]
***********************************
rational number:
(5, 15)
continued fraction:
[0, 3]
convergents:
[(0, 1), (0, 1)]
***********************************
rational number:
(27, 73)
continued fraction:
[0, 2, 1, 2, 2, 1, 2]
convergents:
[(0, 1), (0, 1), (1, 2), (1, 3), (3, 8), (7, 19), (10, 27)]
***********************************
rational number:
(73, 27)
continued fraction:
[2, 1, 2, 2, 1, 2]
convergents:
[(0, 1), (2, 1), (3, 1), (8, 3), (19, 7), (27, 10)]
***********************************


Генерация простых чисел при помощи алгоритма Миллера-Рабина

https://python.su/forum/topic/12364/?page=2#post-77081v

In [8]:
def miller_rabin_pass(a, s, d, n):

    a_to_power = pow(a, d, n)
    i=0
    #Invariant: a_to_power = a^(d*2^i) mod n
    
    # we test whether (a^d) = 1 mod n
    if a_to_power == 1:
        return True

    # we test whether a^(d*2^i) = n-1 mod for 0<=i<=s-1
    while(i < s-1):
        if a_to_power == n - 1:
            return True
        a_to_power = (a_to_power * a_to_power) % n
        i+=1

    # we reach here if the test failed until i=s-2
    return a_to_power == n - 1


def miller_rabin(n):

    #Compute s and d such that n-1 = (2^s)d, with d odd
    d = n-1
    s = 0
    while d%2 == 0:
        d >>= 1
        s+=1

    #Applies the test K times
    #The probability of a false positive is less than (1/4)^K
    K = 20

    i=1
    while(i<=K):
    # 1 < a < n-1
        a = random.randrange(2,n-1)
        if not miller_rabin_pass(a, s, d, n):
            return False
        i += 1

    return True


def gen_prime(nbits):

    while True:
        p = random.getrandbits(nbits)
        #force p to have nbits and be odd
        p |= 2**nbits | 1
        if miller_rabin(p):
            return p
            break

def gen_prime_range(start, stop):

    while True:
        p = random.randrange(start,stop-1)
        p |= 1
        if miller_rabin(p):
            return p
            break
            
def test_MR(name, n):
    
    if name == "test":
        print (miller_rabin(n) and "PRIME" or "COMPOSITE")
    elif name == "genprime":
        nbits = int(n)
        print(gen_prime(nbits))

In [9]:
test_MR('test', 1669)

PRIME


In [10]:
test_MR('test', 2002)

COMPOSITE


In [11]:
test_MR('genprime', 1933)

1526994787345928807504693635272154451275872618337929898035835779697973374876004765913317462343283157621368737570007451722856062025353314869490426138340314529645057301213852602577678015677847598344297448153388743865117967388498863403638125673305250195476534906111913723765587055857888734964281098764575737271476706858972628138722692267574403253784742184932853700905019130272189388131699577492205651164547617646509509783024924591194230309423887995386404395315705227261935533507273130376156082960754337324022903660571988887468981148782119493936666246825389856089085923378573124645024307


Генерация ключей системы RSA и Рабина

In [14]:
def getPrimePair(bits=512):

    assert bits%4==0
    
    p = gen_prime(bits)
    q = gen_prime_range(p+1, 2*p)
    
    return p,q

def generateKeysRSA(nbits=1024):
    '''
    Генерация пары ключей:
        public = (e,n)
        private = d 
    при этом n имеет длину nbits
    '''
    # nbits >= 1024 is recommended
    assert nbits%4==0
    
    p,q = getPrimePair(nbits//2)
    n = p*q
    phi = totient(p, q)
        
    # generate a d such that:
    #     (d,n) = 1
    good_d = False
    while not good_d:
        d = random.getrandbits(nbits//4)
        if (gcd(d,phi) == 1):
            good_d = True
                    
    e = modInverse(d,phi)
    return e,n,d


def generateKeysRabin(nbits=1024):
    
    # nbits >= 1024 is recommended
    assert nbits%4==0
    
    p,q = getPrimePair(nbits//2)
    n = p*q
    
    return n, p, q
    

def cipherRabinKey(nbits=1024, m):
    
    e,n,d = generateKeysRSA(256)
    
    if m >= n:
        e,n,d = generateKeysRSA(256)
    
    return c = pow(m, e, n)
    
    
    
    

def testkeys():
    
    for i in range(2):
        e,n,d = generateKeysRSA()
        print ("Public key:")
        print("e =")
        print(e)
        print("n =")
        print(n)
        print ("Private key:")
        print("d =")
        print(d)
        print("-----------------------")

In [15]:
testkeys()

Public key:
e =
352830055678491448173214660652418401221577847109156185024045784559385630042900817031095569704987094038440391062595506309943142523909510880720687815889424151530315398811103937329582098535237085489709105534292565132689968649112801891708889052400108358549934508940099752140732315543649643640899246203092415072093
n =
435953454432121489368948165843115325378613054144963301578205938821674762805409961540525759378239220145769207540609427307336396898799592613180035323959715265234072687222246923891640901974344321591367807254431414221963298951587125811789910193771630553883059393863744225621417723487115049256570996433548836786097
Private key:
d =
95436847667104629643289876026791578721667825544379903952513516880864603910293
-----------------------
Public key:
e =
322891192362572093119622833049802971199586884527454293047659945294588876413503865700667288161284213017226199788917329297083168066168187274905070423059781150393164949832893457127137572110940460644492996447639684400359265175548